In [1]:
import pyodbc
import pandas as pd

# Anslut till SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MUSTAF;DATABASE=CrimeNotificationSystem;Trusted_Connection=yes;')

# Hämta brottshändelser från SQL Server
query = "SELECT Latitude, Longitude, EventName FROM CrimeEvents WHERE EventDate > '2024-10-01'"
df = pd.read_sql(query, conn)

# Stäng anslutningen
conn.close()

# Kontrollera att data har hämtats korrekt
print(df.head())

    Latitude  Longitude       EventName
0  59.376712  16.507310           Brand
1  59.284121  15.229580  Trafikkontroll
2  57.726385  11.945879  Trafikkontroll
3  56.640868  12.904317  Trafikkontroll
4  59.170904  17.640408  Trafikkontroll


C:\Users\musta\AppData\Local\Temp\ipykernel_21172\2810312726.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:



import folium

# Skapa en karta över Sverige
m = folium.Map(location=[60.128161, 18.643501], zoom_start=5)

# Lägg till markörer för varje brottshändelse
for index, row in df.iterrows():
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=row['EventName']
    ).add_to(m)

# Spara kartan till en HTML-fil
m.save('brottskarta.html')


In [4]:
from folium.plugins import HeatMap

# Skapa en lista med koordinater från DataFrame
heat_data = [[row['Latitude'], row['Longitude']] for index, row in df.iterrows()]

# Skapa en heatmap på en karta
m = folium.Map(location=[60.128161, 18.643501], zoom_start=5)

# Lägg till HeatMap-lagret på kartan
HeatMap(heat_data).add_to(m)

# Spara kartan som en HTML-fil
m.save('brott_heatmap.html')


In [2]:
import folium
from folium.plugins import MarkerCluster
import pyodbc
import pandas as pd

# Anslut till SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MUSTAF;DATABASE=CrimeNotificationSystem;Trusted_Connection=yes;')

# Hämta brottshändelser från SQL Server
query = "SELECT Latitude, Longitude, EventName, Location FROM CrimeEvents WHERE EventDate > '2024-10-01'"
df = pd.read_sql(query, conn)

# Stäng anslutningen till SQL Server
conn.close()

# Skapa en karta över Sverige
m = folium.Map(location=[60.128161, 18.643501], zoom_start=5)

# Skapa en markercluster för att gruppera markörer
marker_cluster = MarkerCluster().add_to(m)

# Skapa en dictionary för att hantera olika brottstyper
crime_type_groups = {}

# Loopa igenom DataFrame och lägg till markörer i rätt grupp baserat på brottstyp
for index, row in df.iterrows():
    crime_type = row['EventName']  # Brottstyp
    city = row['Location']  # Stad
    
    # Om brottstypen inte redan finns i dictionaryn, skapa en ny FeatureGroup
    if crime_type not in crime_type_groups:
        crime_type_groups[crime_type] = folium.FeatureGroup(name=crime_type)
    
    # Skapa en markör för varje händelse
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['EventName']} - {row['Location']}",
        tooltip=row['EventName']
    ).add_to(crime_type_groups[crime_type])

# Lägg till varje brottstyp till kartan
for group in crime_type_groups.values():
    group.add_to(m)

# Lägg till en LayerControl för att hantera visning och dölja olika brottstyper
folium.LayerControl().add_to(m)

# Spara kartan till en HTML-fil
m.save('brottskarta_med_filter.html')


C:\Users\musta\AppData\Local\Temp\ipykernel_41500\2095597229.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [14]:
from folium.plugins import MarkerCluster
import folium
import pyodbc
import pandas as pd

# Anslut till SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MUSTAF;DATABASE=CrimeNotificationSystem;Trusted_Connection=yes;')

# Hämta brottshändelser från SQL Server
query = "SELECT Latitude, Longitude, EventName FROM CrimeEvents WHERE EventDate > '2024-10-01'"
df = pd.read_sql(query, conn)

# Stäng anslutningen till SQL Server
conn.close()

# Skapa en karta över Sverige
m = folium.Map(location=[60.128161, 18.643501], zoom_start=5)

# Skapa en MarkerCluster för att hantera markörerna
marker_cluster = MarkerCluster().add_to(m)

# Lägg till markörer i klustret baserat på brottshändelser
for index, row in df.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['EventName']}",
            tooltip=row['EventName']
        ).add_to(marker_cluster)

# Spara kartan som en HTML-fil
m.save('brottskarta_med_markercluster.html')


C:\Users\musta\AppData\Local\Temp\ipykernel_41500\2530573083.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [16]:
import folium
from folium.plugins import HeatMap
import pyodbc
import pandas as pd

# Anslut till SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MUSTAF;DATABASE=CrimeNotificationSystem;Trusted_Connection=yes;')

# Hämta brottshändelser från SQL Server
query = "SELECT Latitude, Longitude, EventName FROM CrimeEvents WHERE EventDate > '2024-10-01'"
df = pd.read_sql(query, conn)

# Stäng anslutningen till SQL Server
conn.close()

# Skapa en karta över Sverige
m = folium.Map(location=[60.128161, 18.643501], zoom_start=5)

# Dela upp brottstyper i kategorier, ex: Våldsbrott, Egendomsbrott, Trafikbrott, Övriga incidenter
violence_crimes = df[df['EventName'].isin(['Bråk', 'Misshandel', 'Rån', 'Våldtäkt', 'Mord/dråp', 'Våld/hot mot tjänsteman', 'Olaga hot', 'Skottlossning', 'Ofredande/förargelse'])]  # Våldsbrott
property_crimes = df[df['EventName'].isin(['Stöld', 'Inbrott', 'Bedrägeri', 'Motorfordon', 'Skadegörelse', 'Åldringsbrott', 'Olaga intrång', 'Stöld/inbrott', 'Hemfridsbrott', 'Häleri'])]  # Egendomsbrott
traffic_crimes = df[df['EventName'].isin(['Trafikbrott', 'Trafikkontroll', 'Trafikolycka', 'Trafikhinder', 'Rattfylleri', 'Olovlig körning', 'Knivlagen', 'Vapenlagen'])]  # Trafikrelaterade brott
other_incidents = df[df['EventName'].isin(['Anträffad död', 'Arbetsplatsolycka', 'Brand', 'Djur', 'Explosion', 'Farligt föremål', 'Fjällräddning', 'Fylleri/LOB', 'Försvunnen person', 'Kontroll person/fordon', 'Larm Inbrott', 'Polisinsats/kommendering', 'Sammanfattning kväll och natt', 'Sammanfattning natt', 'Övrigt'])]  # Övriga incidenter och insatser

# Skapa heatmap-data för varje kategori
violence_heat_data = [[row['Latitude'], row['Longitude']] for index, row in violence_crimes.iterrows() if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude'])]
property_heat_data = [[row['Latitude'], row['Longitude']] for index, row in property_crimes.iterrows() if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude'])]
traffic_heat_data = [[row['Latitude'], row['Longitude']] for index, row in traffic_crimes.iterrows() if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude'])]
other_incidents_heat_data = [[row['Latitude'], row['Longitude']] for index, row in other_incidents.iterrows() if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude'])]

# Skapa olika FeatureGroups för olika brottstyper
violence_heatmap = folium.FeatureGroup(name='Våldsbrott Heatmap')
property_heatmap = folium.FeatureGroup(name='Egendomsbrott Heatmap')
traffic_heatmap = folium.FeatureGroup(name='Trafikrelaterade brott Heatmap')
other_incidents_heatmap = folium.FeatureGroup(name='Övriga incidenter Heatmap')

# Lägg till HeatMap-lager för varje kategori
HeatMap(violence_heat_data, name="Våldsbrott", min_opacity=0.2, max_zoom=18, radius=15, blur=10, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(violence_heatmap)
HeatMap(property_heat_data, name="Egendomsbrott", min_opacity=0.2, max_zoom=18, radius=15, blur=10, gradient={0.4: 'green', 0.65: 'yellow', 1: 'red'}).add_to(property_heatmap)
HeatMap(traffic_heat_data, name="Trafikrelaterade brott", min_opacity=0.2, max_zoom=18, radius=15, blur=10, gradient={0.4: 'orange', 0.65: 'purple', 1: 'red'}).add_to(traffic_heatmap)
HeatMap(other_incidents_heat_data, name="Övriga incidenter", min_opacity=0.2, max_zoom=18, radius=15, blur=10, gradient={0.4: 'pink', 0.65: 'yellow', 1: 'red'}).add_to(other_incidents_heatmap)

# Lägg till alla heatmaps till kartan
violence_heatmap.add_to(m)
property_heatmap.add_to(m)
traffic_heatmap.add_to(m)
other_incidents_heatmap.add_to(m)

# Lägg till en LayerControl för att användaren ska kunna slå av och på olika lager
folium.LayerControl().add_to(m)

# Spara kartan som en HTML-fil
m.save('brottskarta_färfilt.html')


C:\Users\musta\AppData\Local\Temp\ipykernel_41500\1335637119.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
